In [25]:
import pandas as pd
import ssl 
import os
import logging
from pathlib import Path
import time


ssl._create_default_https_context = ssl._create_unverified_context

Table: 'career_avg_bowler'

In [26]:
def career_avg_bowler(player_ids):

    career_avg_bowler_merged = pd.DataFrame()
    
    for p_id in player_ids:
        
        career_avg_bowler = pd.read_html("https://stats.espncricinfo.com/ci/engine/player/"+p_id+".html?class=2;template=results;type=bowling")[2]
        career_avg_bowler=career_avg_bowler[["Span","Inns","Overs","Mdns","Runs","Wkts","Ave","Econ","SR"]]
        career_avg_bowler.insert(0, "P_id", [p_id], True)

        dismissal = pd.read_html("https://stats.espncricinfo.com/ci/engine/player/"+p_id+".html?class=2;template=results;type=bowling;view=dismissal_summary")[-4]
        dismissal_types=["caught", "bowled", "leg before wicket"]
        dismissal.set_index("Grouping", inplace = True)

        #avail_types=[]
        #non_avail_types=[]
        #for index, row in dismissal.iterrows():
        #    if index in types:
        #        avail_types.append(index)
        #non_avail_types = np.setdiff1d(types, avail_types)

        dismissal=dismissal.loc[dismissal_types,["Dis"]]
        dismissal_dict = dismissal.to_dict()
        dismissal = pd.DataFrame.from_dict(dismissal_dict.values())

        career_avg_bowler = pd.concat([career_avg_bowler, dismissal], axis=1)

        #career_avg_bowler[non_avail_types] = [None] 
        career_avg_bowler_merged = pd.concat([career_avg_bowler_merged, career_avg_bowler], ignore_index=True)
        time.sleep(0.1)

    filepath = Path('Csv/Career_avg_bowler.csv')
    career_avg_bowler_merged.to_csv(filepath, index_label="row_id")  
    print(career_avg_bowler_merged)


Table: 'vs_country_bowler'

In [27]:
def vs_country_bowler(player_ids):

    vs_country_bowler_merged = pd.DataFrame()
 
    for p_id in player_ids:
        
        vs_country_bowler = pd.read_html("https://stats.espncricinfo.com/ci/engine/player/"+p_id+".html?class=2;template=results;type=bowling")[3]
        vs_country_bowler=vs_country_bowler[['Grouping','Inns', 'Overs','Mdns','Runs','Wkts','Ave','Econ','SR']]

        vs_country_bowler.rename(columns = {'Grouping':'Country'}, inplace = True)
        vs_country_bowler.insert(0, "P_id", p_id, True)

        countries=['v Pakistan', 'v Australia', 'v England', 'v New Zealand', 'v West Indies', 'v Sri Lanka', 'v Bangladesh', 'v South Africa']

        vs_country_bowler = vs_country_bowler.loc[ vs_country_bowler['Country'].isin(countries)  ] 
        vs_country_bowler_merged = pd.concat([vs_country_bowler_merged, vs_country_bowler], ignore_index=True)
        time.sleep(0.1)


    filepath = Path('Csv/Vs_country_bowler.csv')
    vs_country_bowler_merged.to_csv(filepath, index_label="row_id")  
    print(vs_country_bowler_merged)

Table: 'home_vs_away_bowler'

In [28]:
def home_vs_away_bowler(player_ids):

    home_vs_away_bowler_merged = pd.DataFrame()
 
    for p_id in player_ids:

        home_vs_away_bowler = pd.read_html("https://stats.espncricinfo.com/ci/engine/player/"+p_id+".html?class=2;template=results;type=bowling")[3]
        home_vs_away_bowler=home_vs_away_bowler[['Grouping','Inns', 'Overs','Mdns','Runs','Wkts','Ave','Econ','SR']]

        home_vs_away_bowler.rename(columns = {'Grouping':'Venue'}, inplace = True)
        home_vs_away_bowler.insert(0, "P_id", p_id, True)

        venues=['home', 'away', 'neutral']

        home_vs_away_bowler = home_vs_away_bowler.loc[ home_vs_away_bowler['Venue'].isin(venues)  ] 
        home_vs_away_bowler_merged = pd.concat([home_vs_away_bowler_merged, home_vs_away_bowler], ignore_index=True)
        time.sleep(0.1)

    filepath = Path('Csv/Home_vs_away_bowler.csv')
    home_vs_away_bowler_merged.to_csv(filepath, index_label="row_id")  
    print(home_vs_away_bowler_merged)

Table: 'yearly_stats_bowler'

In [29]:
def yearly_stats_bowler(player_ids):

    yearly_stats_bowler_merged = pd.DataFrame()

    for p_id in player_ids:
       
        yearly_stats_bowler = pd.read_html("https://stats.espncricinfo.com/ci/engine/player/"+p_id+".html?class=2;template=results;type=bowling")[3]
        yearly_stats_bowler=yearly_stats_bowler[['Grouping','Inns', 'Overs','Mdns','Runs','Wkts','Ave','Econ','SR']]

        yearly_stats_bowler.rename(columns = {'Grouping':'Year'}, inplace = True)
        yearly_stats_bowler.insert(0, "P_id", p_id, True)

        years=['home', 'away', 'neutral']

        yearly_stats_bowler = yearly_stats_bowler.loc[ yearly_stats_bowler['Year'].str.startswith('year ', na=False)  ] 
        yearly_stats_bowler_merged = pd.concat([yearly_stats_bowler_merged, yearly_stats_bowler], ignore_index=True)
        time.sleep(0.1)


    filepath = Path('Csv/Yearly_stats_bowler.csv')
    yearly_stats_bowler_merged.to_csv(filepath, index_label="row_id")  
    print(yearly_stats_bowler_merged)

#Main function below:

In [30]:
#234675: Ravindrasinh Anirudhsinh Jadeja (ALL)
#625371: Hardik Himanshu Pandya (ALL)
#26421: Ravichandran Ashwin (ALL)
#625383: Jasprit Jasbirsingh Bumrah (BOW)
#481896: Mohammed Shami Ahmed (BOW)
#326016: Bhuvneshwar Kumar Singh (BOW)
#376116: Umeshkumar Tilak Yadav (BOW)

player_ids=['234675', '625371', '26421', '625383', '481896', '326016', '376116']

os.system("rm -f Csv/*_bowler.csv")

career_avg_bowler(player_ids)
vs_country_bowler(player_ids)
home_vs_away_bowler(player_ids)
yearly_stats_bowler(player_ids)


     P_id       Span  Inns   Overs  Mdns  Runs  Wkts    Ave  Econ    SR  \
0  234675  2009-2023   184  1586.2    55  7744   212  36.52  4.88  44.8   
1  625371  2016-2023    80   533.1    15  2960    84  35.23  5.55  38.0   
2   26421  2010-2023   114  1050.3    37  5180   156  33.20  4.93  40.4   
3  625383  2016-2023    83   725.2    52  3347   143  23.40  4.61  30.4   
4  481896  2013-2023    95   799.0    49  4437   180  24.65  5.55  26.6   
5  326016  2012-2022   120   974.3    68  4951   141  35.11  5.08  41.4   
6  376116  2010-2018    73   593.0    23  3565   106  33.63  6.01  33.5   

   caught  bowled  leg before wicket  
0   100.0    54.0               36.0  
1    71.0     8.0                5.0  
2    84.0    29.0               25.0  
3    88.0    43.0               12.0  
4   110.0    60.0               10.0  
5   105.0    26.0               10.0  
6    76.0    20.0               10.0  
      P_id         Country Inns  Overs Mdns  Runs Wkts    Ave  Econ    SR
0   234675   